# Machine Learning Assignment 4 - Logistic Regression
Course code : CMPE-257 
Group name : Seekers 
Members :

               * Anuradha Rajashekar(012409956)
               * Ashwini Shankar Narayan(012506910)
               * Nidhi Jamar(010070593)
               * Sindhu Goudru Shivanandappa Patil(010823683)
               
## 1. Data Story  
The data in the dataset is from a real Czech bank from 1999. The data is about clients and their accounts and has relation account, client, disposiiton,permanent order, transaction, loan, credit card and demographic data. Relations 'loans' and 'credit card' describe some services which bank offers to its clients. More credit cards can be issued to an account but atmost one loan can be granted for an account.
The business objective is to analyze the transactional behavior of the customers and predict the loan approval status for the same customers.This is useful in cases where credit card companies provide pre approved loans for customers based on their payment and other factors. To acheive this, the below mentioned steps are followed,

1. Data preparation and adding account details - Nidhi Jamar
2. Identifying the categorical value from the features in dataset - Ashwini Shankar Narayan
3. Applying Logistic Regression on the identified column - Sindhu Goudru Shivanandappa Patil
4. Predicting the accuracy - Anuradha Rajashekar

## 2. Dataset Selection
Dataset Link : https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions

## 3. Data Preparation

In data preparation step, we read and preprocess the data to identify and remove invlaid values such as ? or NA. Here, we read and parse all the csv files in the dataset using the pandas dataframe.Pandas provide a unique advantage over other libraries in preprocessing the data by providing inbuilt APIs for all the math operations on selected row/column or full dataset. If there are any invalid values, fill it with the median value of the column.

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [24]:
trans_d = pd.read_csv("data/trans.csv")
loan_d = pd.read_csv("data/loan.csv")
account_d = pd.read_csv("data/account.csv")
client_d = pd.read_csv("data/client.csv")
disp_d = pd.read_csv("data/disp.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Add the details about when the account was created.
The loan dataset contains only loan details with accountid. No other information of the client is present in loan dataset. Therefore, we are adding account creation date, account login frequency, date of birth of the client and the district id - loaction of the branch to the loan dataset.
The output of this section shows that there are no missing/ invalid values.


In [25]:
# Add information about when the account was created
_column_date = []
_column_frequency = []
_column_district = []
_column_clientdob =[]
for c in loan_d['account_id']:
    _result = account_d.loc[account_d['account_id'] == c]
    _column_date.append(_result['date'].iloc[0])
    _column_frequency.append(_result['frequency'].iloc[0])
    _column_district.append(_result['district_id'].iloc[0])
    _result = disp_d.loc[disp_d['account_id'] == c]
    _result = client_d.loc[client_d['client_id'] == _result['client_id'].iloc[0]]
    _column_clientdob.append(_result['birth_number'].iloc[0])
    
loan_d['account_date'] = _column_date
loan_d['account_loginfrequency'] = _column_frequency
loan_d['account_district'] = _column_district
loan_d['client_birthNumber'] = _column_clientdob




missing = loan_d.isna().sum()
print(missing)

loan_id                   0
account_id                0
date                      0
amount                    0
duration                  0
payments                  0
status                    0
account_date              0
account_loginfrequency    0
account_district          0
client_birthNumber        0
dtype: int64


## 4. Identifying a categorical value form one of the columns in our dataset

### Here, we are getting to know the possible unique values for status feature in loan dataset and login frequency in account dataset.
The only two features with String values from all the datasets is 'status' from loan and 'account_loginfrequency'. Therefore, we need to convert them to integer values. 
From the output, we understand 'status' feature has 4 unique values: 'A', 'B', 'C' and 'D',
where A stands for contract finished, no problem,
      B stands for contract finished, loan not payed,
      C stands for running contract, OK so far and
      D stands for running contract, client in debt
  'account_loginfrequency' - frequency of issuance of statements,  has 3 unique values 'POPLATEK TYDNE', 'POPLATEK MESICNE', 'POPLATEK PO OBRATU'   
  where "POPLATEK TYDNE" stands for monthly issuance 
        "POPLATEK TYDNE" stands for weekly issuance 
        "POPLATEK PO OBRATU" stands for issuance after transaction 
 
Here, we are using index values of these feature tuples to set an integer value. For example, in 'status' feature class B willbe assigned integer value 0.

The model is not able to differentiate between clients who have finished contract and who havenot finished contract clearly beacuse there is no column in loan dataset which says whether the contract is finished or not. Therefore, We are reducing the number of classes to binary i.e two classes by classifying as clients who are paying loan regularly and who are not paying regularly.       

In [26]:
_statusUnique = list(loan_d['status'].unique())
_freqUnique = list(loan_d['account_loginfrequency'].unique())
print('Unique values in status :{}'.format(_statusUnique))
print('Unique values in account login frequency :{}'.format(_freqUnique))
loan_d['status'] = [_statusUnique.index(item) for item in loan_d['status']] 
loan_d['account_loginfrequency'] = [_freqUnique.index(item) for item in loan_d['account_loginfrequency']] 
_temp = []
for d in loan_d['status']:
    if(d==0 or d==3):
        _temp.append(0)
    else:
        _temp.append(1)
loan_d['binary_status']  = _temp        
    

Unique values in status :['B', 'A', 'C', 'D']
Unique values in account login frequency :['POPLATEK TYDNE', 'POPLATEK MESICNE', 'POPLATEK PO OBRATU']


Here we are splitting the dataset into training data and testing data. We are considering 70% of the data for training and 30% of the data to test and predicting the value for column 'status'. We are splitting the dataset into training data and testing data both for 4 class classification of 'status' and binary classification of 'status' feature.

In [27]:
from sklearn.cross_validation import train_test_split
cols = list(loan_d.columns)
cols.remove('loan_id')
cols.remove('account_id')
cols.remove('status')
cols.remove('binary_status')
X = loan_d[cols].iloc[ :, 0:].values
Y = loan_d['status']
Y_bin = loan_d['binary_status']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X, Y_bin, test_size=0.3, random_state=0)

## 5. Applying Logistic Regression on 'status' feature
In this step, we are trying to predict the value of 'status' feature by applying Logistic Regression on a random value  in test dataset. For the test dataset, after applying logistic regression, the ground truth value and predicted value is 1.

In [28]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
logreg.fit(X_train, y_train)

logreg_bin = LogisticRegression(C=1e5)
logreg_bin.fit(X_train_bin, y_train_bin)

num = np.random.randint(low=0, high=np.shape(X_test)[0], size=1)[0]
result=logreg.predict(X_test[num,:].reshape(1, -1))[0]
gt= y_test[num]
print('4 class logistic regression   index : ',num,', predicted : ',result,', Ground truth : ',gt)


4 class logistic regression   index :  196 , predicted :  2 , Ground truth :  2


In [37]:
num = np.random.randint(low=0, high=np.shape(X_test_bin)[0], size=1)[0]
result=logreg_bin.predict(X_test_bin[num,:].reshape(1, -1))[0]
gt= y_test_bin[num]
print(' Binary class logistic regression   index : ',num,', predicted : ',result,', Ground truth : ',gt)

 Binary class logistic regression   index :  103 , predicted :  1 , Ground truth :  1


## 6. Predicting the precision and recall for the column 'status'
After the machine is trained with 70% dataset, we are trying to predict the accuracy of the test data upon appying Logistic Regression. We can observe, for 4 class 'status' feature, the precision is 60% and recall is 67%. This is beacuse the model was not able to clearly distinguish between clients who belong to classes 'A' and 'C'- who are paying loan regularly and classes 'B' and 'D' - who are not paying loan regulalry.

### Printing the classification report and confusion matrix.
The classification report consists of precision, recall, f1- score and support. 

In [45]:
from sklearn.metrics import classification_report
Y_pred = logreg.predict(X_test)
print('Metrics for 4 class logistic regression\n',classification_report(y_test, Y_pred))


Metrics for 4 class logistic regression
              precision    recall  f1-score   support

          0       0.00      0.00      0.00        11
          1       0.70      0.50      0.58        62
          2       0.67      0.91      0.78       117
          3       0.00      0.00      0.00        15

avg / total       0.60      0.67      0.62       205



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


After making 'status' feature as binary, the precision increased to 76% and recall increased to 87% meaning the model is now able to clearly distinguish between clients paying loan regularly and not paying.

In [47]:
from sklearn.metrics import classification_report
Y_pred_bin = logreg_bin.predict(X_test_bin)
print('Metrics for binary class logistic regression\n',classification_report(y_test_bin, Y_pred_bin))



Metrics for binary class logistic regression
              precision    recall  f1-score   support

          0       0.00      0.00      0.00        26
          1       0.87      1.00      0.93       179

avg / total       0.76      0.87      0.81       205



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The confusion matrix gives the true positives, true negatives, false positives and false negatives. The confusion matrix of binary class logistic regression has less false positives compared to 4 class logistic regression.

In [48]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, Y_pred)
print('Confusion matrix of 4 class logistic regression\n',confusion_matrix)

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test_bin, Y_pred_bin)
print('\n\nConfusion matrix of binary class logistic regression\n',confusion_matrix)

Confusion matrix of 4 class logistic regression
 [[  0   5   6   0]
 [  0  31  31   0]
 [  0   8 107   2]
 [  0   0  15   0]]


Confusion matrix of binary class logistic regression
 [[  0  26]
 [  0 179]]


### Discuss why you chose that attribute/feature and what you are trying to learn from the classification.

Logistic Regression needs a categorical value to do the classificcation. Our dataset has only 2 features with categorical vlaues and 'status' feature in loan dataset and 'account_loginfrequency' feature in account dataset. Status feature is a good candidate to be considered for classification which predicts to which clients loan approval can be done and for whom not to approve the loan. The classification done for loan status basically helps banks or credit card companies to predict candidates for loan approval.

#### How does this further develop the story you are telling or discovering about your data set and business problem you are trying to solve?

By classifying clients who pay loan regularly and who do not, banks or credit card companies can predict the defaulters for loan payment and approve the loan only to customers are pay regularly.
Our business objective is to give promotions to clients based on their payments and other factors. In such cases, this classification helps in identifying clients for loan pre-approval who are not defaulters.

